In [1247]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [1249]:
def prepareimage(path, target_size=(600, 800)):
    img = Image.open(path).convert('L') 
    img = img.resize(target_size)
    img_array = np.array(img) / 255.0 
    return img_array.flatten()  

In [1251]:
#red neuronal
class NN:
    def __init__(self, input_size, hidden_size):
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2./input_size)
        self.b1 = np.zeros(hidden_size)
        self.W2 = np.random.randn(hidden_size, 1) * np.sqrt(2./hidden_size)
        self.b2 = np.zeros(1)
        self.dropout_rate = 0.4
        self.l2_lambda = 0.01 
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def forward(self, x, is_training=True):
        self.z1 = np.dot(x, self.W1) + self.b1
        self.a1 = self.relu(self.z1)
        
        if is_training:
            self.mask = (np.random.rand(*self.a1.shape) > self.dropout_rate)
            self.a1 *= self.mask / (1 - self.dropout_rate)
        
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        return self.sigmoid(self.z2)
    
    def backpropagation(self, x, y, learning_rate):
        m = x.shape[0]

        d_z2 = self.a2 - y.reshape(-1, 1)
        d_W2 = (1/m) * np.dot(self.a1.T, d_z2) + (self.l2_lambda * self.W2)
        d_b2 = (1/m) * np.sum(d_z2, axis=0)
        
        d_a1 = np.dot(d_z2, self.W2.T)
        if hasattr(self, 'mask'):
            d_a1 *= self.mask
        d_z1 = d_a1 * (self.z1 > 0)  #derivada relu
        
        d_W1 = (1/m) * np.dot(x.T, d_z1) + (self.l2_lambda * self.W1)
        d_b1 = (1/m) * np.sum(d_z1, axis=0)
        
        self.W1 -= learning_rate * d_W1
        self.b1 -= learning_rate * d_b1
        self.W2 -= learning_rate * d_W2
        self.b2 -= learning_rate * d_b2

In [1253]:
X_train = [
    load_and_preprocess("sammy1.jpg"),
    load_and_preprocess("model1.jpg"),
    load_and_preprocess("model2.jpg")
]
y_train = np.array([1, 0, 0])

In [1255]:
#entrenamiento
def train(model, X, y, epochs=1000, lr=0.0001):
    loss_history = []
    X = np.array(X)
    
    for epoch in range(epochs):
        
        model.a2 = model.forward(X)
        loss = -np.mean(y * np.log(model.a2) + (1-y) * np.log(1-model.a2))
        loss_history.append(loss)
        
        model.backpropagation(X, y, lr)
        
        #early stop si no hay mejora
        if epoch > 50 and (loss_history[-50] - loss) < 1e-5:
            print(f"stop en epoch {epoch}")
            break
        
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}")

In [1257]:
input_size = 600 * 800  
model = NN(input_size, hidden_size=128)  # cantidad de neuronas ocultas
train(model, X_train, y_train, epochs=1000, lr=0.0001)

Epoch 0, Loss: 0.7738
stop en epoch 51


In [1287]:
#pruebas
def predict(image_path, model):
    img = prepareimage(image_path)
    pred = model.forward(img, is_training=False)
    prob = pred[0]
    return f" {prob:.2%}, {'es sammy' if prob > 0.80 else 'no es sammy'}"

In [1289]:
print(predict("sammy1.jpg", model))

 92.29%, es sammy
